In [100]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [101]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [102]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [103]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!')]}

In [104]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [105]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [112]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "",
  "new_lines": "Human: Hi I'm Nicolas, I live in South Korea\nAI: Wow that is so cool!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n\n\nNew lines of conversation:\nHuman:

In [113]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions they live in South Korea. The AI responds with enthusiasm, calling it cool.'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}